In [46]:
import json
from uuid import uuid4
import requests

In [39]:
class GigaTurnIp():
	rest_api = "http://127.0.0.1:8000/"

	def __init__(self, headers):
		self.headers = headers

	def post_request(self, url, data):
		return requests.post(self.rest_api + url, data=data, headers=self.headers)

	def get_request(self, url, data=None):
		if not data:
			data = ''
		return requests.get(self.rest_api + url, data=data, headers=self.headers)

	def patch_request(self, url, data):
		return requests.patch(self.rest_api + url, data=data, headers=self.headers)

	def get_objects(self, url, params=None):
		data = params
		if not data:
			data = ''
		return json.loads(self.get_request(url, data=data).content)

	def create_campaign(self):
		data = {"name": "Initial campaign", "description": "", "managers": []}
		response = json.loads(self.post_request(url="api/v1/campaigns/", data=data).content)
		return self.get_objects(f"api/v1/campaigns/{response.get('id')}/")

	def create_chain(self, campaign_id):
		data = {"name": "Init chain", "description": "", "campaign": campaign_id}
		response = json.loads(self.post_request(url="api/v1/chains/", data=data).content)
		return self.get_objects(f"api/v1/chains/{response.get('id')}/")

	def create_task_stage(self, chain_id):
		data = {
			"name": "Initial",
			"x_pos": 1,
			"y_pos": 1,
			"chain": chain_id
		}
		response = json.loads(self.post_request(url="api/v1/taskstages/", data=data).content)
		return self.get_objects(f"api/v1/taskstages/{response.get('id')}/")

	def create_rank(self, stage_id):
		data = {"name": stage_id, "description": ""}
		response = json.loads(self.post_request(url="api/v1/ranks/", data=data).content)
		return self.get_objects(f"api/v1/ranks/{response.get('id')}/")

	def create_rank_limit(self, rank_id, stage_id):
		data = {
			"rank": rank_id,
			"stage": stage_id,
			"open_limit": 0,
			"total_limit": 0,
			"is_listing_allowed": True,
			"is_creation_open": False,
		}
		response = json.loads(self.post_request(url="api/v1/ranklimits/", data=data).content)
		return self.get_objects(f"api/v1/ranklimits/{response.get('id')}/")

	def create_task(self, stage_id):
		response = json.loads(self.get_request(f'api/v1/taskstages/{stage_id}/create_task/'))
		return self.get_objects(f'api/v1/tasks/{response.get("id")}/')

	def update_task_responses(self, task_id, responses):
		data = {"responses": responses}
		response = self.patch_request(f"api/v1/tasks/{task_id}/", data)
		return self.get_objects(f"api/v1/tasks/{response.get('id')}/")

	def complete_task(self, task_id, responses=None):
		data = responses
		if not data:
			data = {"complete": True}
		else:
			data = {"complete": True, "responses": responses}
		response = self.patch_request(f'api/v1/tasks/{task_id}/', data)
		return self.get_objects(f'api/v1/tasks/{response.get("id")}/')

	def request_assignment(self, task_id):
		url = f'api/v1/tasks/{task_id}/request_assignment/'
		response = self.get_request(url)
		return self.get_objects(f'api/v1/tasks/{response.get("id")}/')

	def add_stages(self, stage_id, name="task", stages=[]):
		if name == "task":
			data = {"in_stages": stages}
			url = "api/v1/taskstages/"
		elif name == "cond":
			data = {"out_stages": stages}
			url = "api/v1/conditionalstages/"
		response = self.patch_request(f"{url}{stage_id}/", data)
		return self.get_objects(f"{url}{response.get('id')}/")


In [9]:
token = "xxx"
headers = {"Authorization": f"JWT {token}"}

In [40]:
giga = GigaTurnIp(headers)

In [19]:
giga.create_chain(1)

{'id': 1, 'name': 'Init chain', 'description': '', 'campaign': 1}

In [27]:
task_stage = giga.create_task_stage(1)
giga.create_task_stage(1)
giga.get_objects("api/v1/taskstages/")

{'count': 4,
 'next': None,
 'previous': None,
 'results': [{'id': 1,
   'name': 'Initial',
   'description': '',
   'chain': {'id': 1, 'name': 'Init chain', 'description': '', 'campaign': 1},
   'in_stages': [],
   'out_stages': [],
   'x_pos': '1.000',
   'y_pos': '1.000',
   'json_schema': None,
   'ui_schema': None,
   'library': '',
   'copy_input': False,
   'allow_multiple_files': False,
   'is_creatable': False,
   'displayed_prev_stages': [],
   'assign_user_by': 'RA',
   'assign_user_from_stage': None,
   'rich_text': None,
   'webhook_address': None,
   'webhook_payload_field': None,
   'webhook_params': None,
   'webhook_response_field': None,
   'allow_go_back': False,
   'allow_release': False},
  {'id': 2,
   'name': 'Initial',
   'description': '',
   'chain': {'id': 1, 'name': 'Init chain', 'description': '', 'campaign': 1},
   'in_stages': [],
   'out_stages': [],
   'x_pos': '1.000',
   'y_pos': '1.000',
   'json_schema': None,
   'ui_schema': None,
   'library': '',

In [38]:
giga.add_in_stages(1, stages=[2,3,4])

{'id': 1,
 'name': 'Initial',
 'description': '',
 'chain': {'id': 1, 'name': 'Init chain', 'description': '', 'campaign': 1},
 'in_stages': [2, 3, 4],
 'out_stages': [],
 'x_pos': '1.000',
 'y_pos': '1.000',
 'json_schema': None,
 'ui_schema': None,
 'library': '',
 'copy_input': False,
 'allow_multiple_files': False,
 'is_creatable': False,
 'displayed_prev_stages': [],
 'assign_user_by': 'RA',
 'assign_user_from_stage': None,
 'rich_text': None,
 'webhook_address': None,
 'webhook_payload_field': None,
 'webhook_params': None,
 'webhook_response_field': None,
 'allow_go_back': False,
 'allow_release': False}